In [123]:
import akshare as ak
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import calendar
import shutil
import os

# 获取板块股票
def get_industry_stocks(industry='白酒') -> pd.DataFrame:
    stocks = ak.stock_board_industry_cons_ths(symbol=industry)

    return pd.DataFrame({
        "code": stocks["代码"],
        "name": stocks["名称"],
        "tradable": stocks["流通股"],
        "market_value": stocks["流通市值"]
    })


# 获取股票价格
def get_stock_price(symbol: str, start_date: datetime, end_date: datetime, adjust: str) -> pd.DataFrame:
    stock_hist = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=start_date.strftime("%Y%m%d"), 
        end_date=end_date.strftime("%Y%m%d"), adjust=adjust)
    
    # print(stock_hist)
    return pd.DataFrame({
        'open': list(stock_hist["开盘"]),
        'close': list(stock_hist["收盘"]),
        'high': list(stock_hist["最高"]),
        'low': list(stock_hist["最低"]),
    }, index=pd.to_datetime(stock_hist["日期"]))

def get_stock_eps(symbol: str) -> pd.DataFrame:
    market = 'sz'
    if (symbol.startswith('00') != True):
        market ='sh'
    stock = "{}{}".format(market, symbol)
    profit_y = ak.stock_profit_sheet_by_yearly_em(symbol=stock)

    eps = pd.Series(list(profit_y["BASIC_EPS"]), index=pd.to_datetime(profit_y["REPORT_DATE"]))

    df_eps = pd.DataFrame({
        "eps": eps,
        })
    df_eps["eps_pct"] = df_eps.sort_index().pct_change()
    # print(df_eps)
    df_eps.fillna(0, inplace=True)

    return df_eps

def get_stock_dividents(symbol: str) -> pd.DataFrame:
    df_dividents = pd.DataFrame()
    dividents_dict = {}

    dividents = ak.stock_dividents_cninfo(symbol=symbol)
    
    # print(dividents["报告时间"])
    dateIndex = dividents["报告时间"].str[:4].apply(lambda x: datetime(int(x), 12, 31) if x is not None else None)
    
    df_dividents = pd.DataFrame({
        "sg": list(dividents["送股比例"]),
        "zz": list(dividents["转增比例"]),
        "px": list(dividents["派息比例"]),
        "desc": list(dividents["实施方案分红说明"]),
        "type": list(dividents["分红类型"])
    }, index = pd.to_datetime(dateIndex))

    df_dividents = df_dividents[df_dividents["type"] == "年度分红"]
    # print(df_dividents)

    df_dividents.index.set_names("REPORT_DATE", inplace=True)
    df_dividents["px_pct"] = df_dividents["px"].sort_index().pct_change()
    df_dividents.fillna(0, inplace=True)  

    return df_dividents

def export_industry_dividents(industry : str):
    stocks = get_industry_stocks(industry)
    # try:
    #     shutil.rmtree(industry)
    #     os.mkdir(industry)
    # except:
    #     os.mkdir(industry)

    start_date = datetime(2013,1,1)
    end_date = datetime.now()

    filepath = f"./{industry}.xlsx"
    xlswriter = pd.ExcelWriter(filepath)

    for i in tqdm(range(len(stocks))):
        code = stocks.loc[i, "code"]
        name = stocks.loc[i, "name"]
        print(code)

        stock_info = ak.stock_individual_info_em(symbol=code)
        # print(stock_info)
        # print(stock_info.loc["上市时间"])
        ipo_date = datetime.strptime(str(stock_info["value"].loc[3]), '%Y%m%d')

        if ipo_date > start_date:
            print('stock ({}) ipo date ({}) is too late'.format(code, ipo_date))
            continue

        df_div = get_stock_dividents(code)
        df_eps = get_stock_eps(code)
        # df_eps_pct = df_eps.pct_change()
        price = get_stock_price(code, start_date=start_date, end_date=end_date, adjust="")
        # print(price["close"][-1])

        df_div["interest"] = df_div["px"] / (price.resample('Y')["close"].last() * 10)

        df = pd.concat([price.resample('Y')["low"].min(),price.resample('Y')["high"].max(), price.resample('Y')["close"].last(), df_eps, df_div], axis=1, join='outer')
        
        df["2010":].to_excel(xlswriter, sheet_name=f'{code}-{name}'.replace('*', ''), index=True)

    xlswriter.close()

In [125]:
export_industry_dividents("白酒")

e:\Python\Python39\lib\site-packages\pandas\compat\_optional.py:161: UserWarning: Pandas requires version '1.2.2' or newer of 'xlsxwriter' (version '1.1.2' currently installed).
  warnings.warn(msg, UserWarning)
  0%|          | 0/20 [00:00<?, ?it/s]

600779


  5%|▌         | 1/20 [00:03<00:59,  3.15s/it]

600199


 10%|█         | 2/20 [00:05<00:53,  2.95s/it]

...

600519


 85%|████████▌ | 17/20 [00:38<00:07,  2.37s/it]

000995


 85%|████████▌ | 17/20 [00:41<00:07,  2.43s/it]


ValueError: Invalid character * found in sheet title

In [19]:
stock_info = ak.stock_individual_info_em(symbol="600036")


In [53]:
print(stock_info)
datetime.strptime(str(stock_info["value"].loc[3]), '%Y%m%d')
# print(stock_info.loc["上市时间"])
    # ipo_date = datetime.strptime(stock_info.loc["上市时间"], 'YYYYMMdd')

   item                value
0   总市值  922037555172.560059
1  流通市值   754194208324.23999
2    行业                   银行
3  上市时间             20020409
4  股票代码               600036
5  股票简称                 招商银行
6   总股本        25219845601.0
7   流通股        20628944429.0


datetime.datetime(2002, 4, 9, 0, 0)

In [95]:
ak.stock_board_industry_name_ths().to_csv("行业.csv")
# price = get_stock_price("001227", start_date=datetime.date(2000,1,1), end_date=datetime.date(2021,12,31), adjust="")